In [5]:
#nbi:hide_in
#nbi:hide_out
!pip3 install matplotlib
!pip3 install keras
!pip3 install spectral
!pip3 install seaborn
!pip3 install scikit-image
!pip3 install tensorflow
!pip3 install sklearn

In [1]:
#nbi:hide_in
%matplotlib inline
import spectral as sp
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from spectrum import rank_normalizer,scale_max_min,scale_white_dark,plot_spectra,zoom
from spectrum_ml import cluster_pixels_kmeans,detect_aphid,blob_detector
from ipywidgets import interact, interactive, widgets
from IPython.display import display

In [2]:
# Create text widget for output
output_slider_variable = widgets.Text()

In [3]:
# Define function to bind value of the input to the output variable 
def f(x):
    output_slider_variable.value = str(x)



In [8]:
# Create input slider with default value = 10    
interact(f, x='adresse')



interactive(children=(Text(value='adresse', description='x'), Output()), _dom_classes=('widget-interact',))

<function __main__.f(x)>

In [9]:
# Display output variable in text box
output_slider_variable



Text(value='adresse')

In [12]:
# Create and output new int variable with value of slider
new_variable = str(output_slider_variable.value)
print(new_variable)



adresse


In [4]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [5]:
def adresse(x):
    return x

In [6]:
exp_name = interactive(adresse, x = "VL1grF")

In [7]:
display(exp_name)

interactive(children=(Text(value='VL1grF', description='x'), Output()), _dom_classes=('widget-interact',))

In [8]:
current_dir = interactive(adresse, x = "/home/plavainne/Documents/BIM/06022020/06022020/")

In [9]:
display(current_dir)

interactive(children=(Text(value='/home/plavainne/Documents/BIM/06022020/06022020/', description='x'), Output(…

In [ ]:
fichier_csv = interactive(adresse, x = "wave_lenghts.csv")

In [11]:
wave_lengths = pd.read_csv(fichier_csv.result,index_col=0,header=None)[1]

NameError: name 'fichier_csv' is not defined

In [ ]:
image_path = current_dir.result+exp_name.result+'/capture/'+exp_name.result+'.hdr'
dark_ref_path = current_dir.result+exp_name.result+'/capture/DARKREF_'+exp_name.result+'.hdr'
white_ref_path = current_dir.result+exp_name.result+'/capture/WHITEREF_'+exp_name.result+'.hdr'